https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

Load Text


In [2]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [4]:
# load document
in_filename = 'republic.txt'
doc = load_doc(in_filename)
print(doc[:200])

BOOK I. The Republic opens with a truly Greek scene--a festival in
honour of the goddess Bendis which is held in the Piraeus; to this is
added the promise of an equestrian torch-race in the evening. T


Clean Text



In [7]:
import string  


In [8]:
 # turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [9]:

# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'the', 'republic', 'opens', 'with', 'a', 'truly', 'greek', 'scene', 'a', 'festival', 'in', 'honour', 'of', 'the', 'goddess', 'bendis', 'which', 'is', 'held', 'in', 'the', 'piraeus', 'to', 'this', 'is', 'added', 'the', 'promise', 'of', 'an', 'equestrian', 'torchrace', 'in', 'the', 'evening', 'the', 'whole', 'work', 'is', 'supposed', 'to', 'be', 'recited', 'by', 'socrates', 'on', 'the', 'day', 'after', 'the', 'festival', 'to', 'a', 'small', 'party', 'consisting', 'of', 'critias', 'timaeus', 'hermocrates', 'and', 'another', 'this', 'we', 'learn', 'from', 'the', 'first', 'words', 'of', 'the', 'timaeus', 'when', 'the', 'rhetorical', 'advantage', 'of', 'reciting', 'the', 'dialogue', 'has', 'been', 'gained', 'the', 'attention', 'is', 'not', 'distracted', 'by', 'any', 'reference', 'to', 'the', 'audience', 'nor', 'is', 'the', 'reader', 'further', 'reminded', 'of', 'the', 'extraordinary', 'length', 'of', 'the', 'narrative', 'of', 'the', 'numerous', 'company', 'three', 'only', 'take

In [10]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 210194


In [11]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()


In [12]:

# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)


In [13]:
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')


In [14]:
print(lines[0])

book i the republic opens with a truly greek scene a festival in honour of the goddess bendis which is held in the piraeus to this is added the promise of an equestrian torchrace in the evening the whole work is supposed to be recited by socrates on the day after


Train Language Model



In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [19]:
print(sequences[0])

[297, 15, 1, 306, 10270, 28, 7, 363, 225, 4604, 7, 1330, 6, 275, 2, 1, 3125, 4603, 11, 5, 850, 6, 1, 2711, 4, 31, 5, 1054, 1, 1765, 2, 49, 6301, 3706, 6, 1, 2710, 1, 147, 278, 5, 402, 4, 10, 6299, 23, 150, 57, 1, 357, 165]


In [22]:
print(max(sequences))

[10270, 28, 7, 363, 225, 4604, 7, 1330, 6, 275, 2, 1, 3125, 4603, 11, 5, 850, 6, 1, 2711, 4, 31, 5, 1054, 1, 1765, 2, 49, 6301, 3706, 6, 1, 2710, 1, 147, 278, 5, 402, 4, 10, 6299, 23, 150, 57, 1, 357, 165, 1, 1330, 4, 7]


In [23]:

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [24]:
print(vocab_size)

10271


In [30]:
from keras.utils import to_categorical
from numpy import array


In [31]:

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]


In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [33]:

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            513550    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10271)             1037371   
Total params: 1,701,821
Trainable params: 1,701,821
Non-trainable params: 0
_________________________________________________________________
None


In [34]:

# compile model
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
#model.fit(X, y, batch_size=128, epochs=100)


//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
210194/210194 [==============================] - 358s 2ms/step - loss: 6.1361 - accuracy: 0.0955
Epoch 2/100
210194/210194 [==============================] - 407s 2ms/step - loss: 5.6557 - accuracy: 0.1327
Epoch 3/100
210194/210194 [==============================] - 410s 2ms/step - loss: 5.4332 - accuracy: 0.1513
Epoch 4/100
210194/210194 [==============================] - 354s 2ms/step - loss: 5.2913 - accuracy: 0.1610
Epoch 5/100
210194/210194 [==============================] - 347s 2ms/step - loss: 5.1750 - accuracy: 0.1670
Epoch 6/100
210194/210194 [==============================] - 337s 2ms/step - loss: 5.0697 - accuracy: 0.1738
Epoch 7/100
210194/210194 [==============================] - 344s 2ms/step - loss: 4.9726 - accuracy: 0.1793
Epoch 8/100
210194/210194 [==============================] - 342s 2ms/step - loss: 4.8860 - accuracy: 0.1827
Epoch 9/100
210194/210194 [==============================] - 348s 2ms/step - loss: 4.8107 - accuracy: 0.1862
Epoch 10/100
210194

In [35]:
from pickle import dump


In [36]:

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))


Load Model


In [39]:
from pickle import load
from keras.models import load_model


In [40]:
# load the model
model = load_model('model.h5')

//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [41]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [66]:

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [67]:
seq_length = len(lines[0].split()) - 1

Generate Text


In [68]:
from random import randint

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

exile or by the hand of the executioner the lesson which he thus receives makes him cautious he leaves politics represses his pride and saves pence avarice is enthroned as his bosoms lord and assumes the style of the great king the rational and spirited elements sit humbly on the ground



In [69]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]
print(encoded)
#encoded = encoded[0 : 50]
#print(len(encoded))

[1604, 13, 23, 1, 305, 2, 1, 3957, 1, 1813, 11, 8, 214, 1715, 324, 45, 5280, 8, 2105, 455, 7088, 25, 1832, 3, 4178, 7089, 2032, 5, 5281, 17, 25, 5282, 1267, 3, 2396, 1, 473, 2, 1, 103, 652, 1, 786, 3, 1211, 578, 1992, 5283, 57, 1, 725]


In [71]:
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

In [72]:
from keras.preprocessing.sequence import pad_sequences

# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=0)


In [74]:

out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

In [75]:

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [76]:

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

into man and in the same way the words of the soul and the afterthought of the ground into the valley in which they are born of prose to the state to be a shadow of the soul and the unjust of the soul and the afterthought of the amatory
